In [12]:
import numpy as np
import pandas as pd

In [13]:
tennis=pd.read_csv('play_tennis.csv')

In [14]:
tennis

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


## valueable_column ()

#### It returns the column name which has the highest information gain

In [72]:
from numpy import log

def log1(yes,no):
    sum1=yes+no
    if yes==0:
        return -(no/sum1*log(no/sum1))
    elif no==0:
        return -(yes/sum1*np.log(yes/sum1))
    return -(yes/sum1*np.log(yes/sum1)+no/sum1*log(no/sum1))


def valueable_column(X):
    info_gain=[]
    yes_count=X[X.columns[len(X.columns)-1]].value_counts()['Yes']
    no_count=X[X.columns[len(X.columns)-1]].value_counts()['No']
    total_entropy=log1(yes=yes_count,no=no_count)
    for i in X.columns[1:-1]:
        name_list=X.groupby(i).count()[X.columns[len(X.columns)-1]].index.tolist()
        no_list=X.groupby(i).count()[X.columns[len(X.columns)-1]].tolist()
        y=0
        for j in range(len(name_list)):
            
            try:
                yes_count1=X[X[i]==name_list[j]][X.columns[len(X.columns)-1]].value_counts()['Yes']
            except:
                yes_count1=0
            try:
                no_count1=X[X[i]==name_list[j]][X.columns[len(X.columns)-1]].value_counts()['No']
            except:
                no_count1=0
            entropy=log1(yes=yes_count1,no=no_count1)
            weight=no_list[j]/sum(no_list)
            y=y+entropy*weight
        info_gain.append(total_entropy-y)
    print(info_gain)
    return X.columns[info_gain.index(max(info_gain))+1]
valueable_column(tennis)

[0.17103394188032706, 0.02025553899523236, 0.10524434967821283, 0.033359115436214726]


'outlook'

# Valuable_columns()    Generalized version

#### This function is a generalized version of valueable_columns. Here, you pass the dataframe, feature_cols=the columns among which you want to get the most valuable columns and the target column. If you don't pass the feature_cols, it will take all the columns except the last column and interpret it as target column

In [73]:
from numpy import log

def log1(unique):
    sum1=sum(unique)
    entropy=0
    for i in unique:
        if i!=0:
            entropy=entropy-i/sum1*np.log(i/sum1)
    return entropy


def valuable_columns(X,feature_cols=None,target=None):
    info_gain=[]
    if feature_cols==None:
        feature_cols=X.columns[0:len(X.columns)-1].tolist()
    if target==None:
        target=X.columns[len(X.columns)-1]
    value_count=X[target].value_counts()
    unique_val_index=value_count.index.tolist()
    unique_val_count=value_count.tolist()
    total_entropy=log1(unique_val_count)
    for i in feature_cols:
        name_list=X.groupby(i).count()[target].index.tolist()
        no_list=X.groupby(i).count()[target].tolist()
        y=0
        for j in range(len(name_list)):
            new_unique_val_count=[]
            for k in unique_val_index:
                try:
                    count=X[X[i]==name_list[j]][target].value_counts()[k]
                    new_unique_val_count.append(count)
                except:
                    new_unique_val_count.append(0)
                    
        
            entropy=log1(new_unique_val_count)
            weight=no_list[j]/sum(no_list)
            y=y+entropy*weight
        info_gain.append(total_entropy-y)
    sort_index=np.argsort(np.array(info_gain)).tolist()
    sort_index.reverse()
    return np.array(feature_cols)[sort_index]


In [74]:
valuable_columns(tennis,['humidity','temp', 'wind','outlook'],'play')

array(['outlook', 'humidity', 'wind', 'temp'], dtype='<U8')

In [75]:
valuable_columns(tennis)

array(['day', 'outlook', 'humidity', 'wind', 'temp'], dtype='<U8')

In [84]:
valuable_columns(tennis,['wind','humidity'],'play')

array(['humidity', 'wind'], dtype='<U8')